一。数据下载

In [ ]:
import hashlib
import os
import tarfile
import zipfile
import requests

DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'


            

In [10]:
def download(name,cache_dir = os.path.join('..','data')):
    assert name in DATA_HUB,f"{name} 不存在于 {DATA_HUB}."
    url,sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir,exist_ok= True)
    fname = os.path.join(cache_dir,url.split('/')[-1])
    if os.path.exists(fname):                     #实际上这是已存在防止重复下载的代码
        sha1 = hashlib.sha1()
        with open(fname,'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname
    print(f'正在从{url}下载{fname}')
    r = requests.get(url,stream =True,verify = True)
    with open(fname,'wb') as f:
        f.write(r.content)
    return fname

In [11]:
def download_extract(name,folder = None):#这是解压函数
    fname = download(name)
    base_dir = os.path.dirname(fname)#找到前一级
    data_dir,ext  = os.path.splitext(fname)   #分割后缀
    if ext == '.zip':
        fp = zipfile.ZipFile(fname,'r')
    elif ext in ('.tar','.gz'):
        fp  = tarfile.open(fname,'r')
    else:
        assert False,'只有zip/tar可以被解压哦'
    fp.extractcall(base_dir) #提取出来
    return os.path.join(base_dir,folder) if folder else data_dir

def download_all():#这是下载函数
    for name in DATA_HUB:
        download(name)
    

In [12]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l 


In [13]:
DATA_HUB['kaggle_house_train'] = (  #@save
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (  #@save
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')


In [14]:
train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))


正在从http://d2l-data.s3-accelerate.amazonaws.com/kaggle_house_pred_train.csv下载..\data\kaggle_house_pred_train.csv
正在从http://d2l-data.s3-accelerate.amazonaws.com/kaggle_house_pred_test.csv下载..\data\kaggle_house_pred_test.csv


In [15]:
print(train_data.shape)
print(test_data.shape)


(1460, 81)
(1459, 80)


In [16]:
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]]) 


   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [19]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))#合并成all_features
print(all_features[0:3,])

   MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0          60       RL         65.0     8450   Pave   NaN      Reg   
1          20       RL         80.0     9600   Pave   NaN      Reg   
2          60       RL         68.0    11250   Pave   NaN      IR1   

  LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC Fence  \
0         Lvl    AllPub    Inside  ...           0        0    NaN   NaN   
1         Lvl    AllPub       FR2  ...           0        0    NaN   NaN   
2         Lvl    AllPub    Inside  ...           0        0    NaN   NaN   

  MiscFeature MiscVal  MoSold  YrSold  SaleType  SaleCondition  
0         NaN       0       2    2008        WD         Normal  
1         NaN       0       5    2007        WD         Normal  
2         NaN       0       9    2008        WD         Normal  

[3 rows x 79 columns]


二。数据预处理